**lab 04: Naive Bayes**

Ta  có $X_i=(x_i^1,x_i^2,...,x_i^n)$ là một sample có n features, với $c$ labels ($C_1,C_2,...C_c$). Ta muốn chọn được label cho sample $X_i$ thì dựa vào:
$$C=\mathop{argmax}_{C_j, j=\overline{1,c}}P(C_j|X_i)$$
Giả sử các thành phần của biến ngẫu nhiên X độc lập với nhau, ta được: 
$$C=\mathop{argmax}_{C_j, j=\overline{1,c}}P(C_j)\prod_{i=1}^n P(X_i|C_j)$$

Tóm lại: với mỗi label $C_j$ ta tính tích $P(C_j)\prod_{i=1}^n P(X_i|C_j)$ và so sánh chúng với nhau từ đó chọn ra được label mang giá trị lớn nhất

Một số phân phối:
1. Multinomial

2. Gaussian


In [1]:
import pandas as pd
import numpy as np

In [2]:
data=pd.read_csv('https://raw.githubusercontent.com/huynhthanh98/ML/master/lab-04/train.csv')
data.head()

,PassengerId,Survived,Pclass,Name,Sex,Age,SibSp,Parch,Ticket,Fare,Cabin,Embarked
0,1,0,3,"Braund, Mr. Owen Harris",male,22.0,1,0,A/5 21171,7.2500,NaN,S
1,2,1,1,"Cumings, Mrs. John Bradley (Florence Briggs Th...",female,38.0,1,0,PC 17599,71.2833,C85,C
2,3,1,3,"Heikkinen, Miss. Laina",female,26.0,0,0,STON/O2. 3101282,7.9250,NaN,S
3,4,1,1,"Futrelle, Mrs. Jacques Heath (Lily May Peel)",female,35.0,1,0,113803,53.1000,C123,S
4,5,0,3,"Allen, Mr. William Henry",male,35.0,0,0,373450,8.0500,NaN,S


Đây là data thống kê các hành khách trên con tàu Titanic, trong đó:
PassengerId là Id hành khách

Survived là sống sót hay không (giá trị dự đoán). 1 là có, 0 là không

Pclass: Loại vé [1,2,3]

Name tên hành khách

Age tuổi hành khách

Parch: Số anh cha mẹ/con trên tàu

SibSp: Số anh chị em/ vợ chông trên tàu

ticket: mã  vé

fare: Phí hành khách

cabin: số khoang

Embarked: Cảng lên tàu

Chúng ta sẽ dự đoán với tất cả features trên thì ai có khả năng sống sót được.


In [3]:
data=data.drop(["Cabin","Age"],axis=1)
data=data.dropna()

In [4]:
from sklearn.model_selection import train_test_split
data_train, data_test = train_test_split( data, test_size=0.2
                                                    , random_state=42)


In [5]:
#Phân tích thử các giá trị có trong mỗi feature
for i in data_train.drop(labels=["Name","Ticket","PassengerId"],axis=1).columns:
  print(i)
  print(data_train[i].unique())

Survived
[1 0]
Pclass
[1 3 2]
Sex
['female' 'male']
SibSp
[0 1 8 5 4 2 3]
Parch
[0 2 1 3 6 4 5]
Fare
[151.55    14.4542   7.925   69.55    46.9     31.275    7.8958  35.5
   0.      26.      23.25    15.5      7.7875  13.5      6.4958  26.55
   7.5208   7.2292  53.1     25.4667  31.3875   8.6625  34.0208   7.125
  52.       7.8875  14.4583  83.1583   8.05    90.       7.725   39.6
  21.075    7.7333   7.25    77.2875   9.5     12.65    10.5     55.
   7.6292   7.75     7.8542  24.15    57.9792  15.      39.6875   7.225
 227.525   17.8      9.5875 113.275    7.8292  13.      30.5      9.
  71.       4.0125   7.05    78.85     8.85    27.75    27.7208 153.4625
   8.3     19.2583  14.5    133.65    76.7292  26.2833  20.525    7.8792
  29.7     15.7417  30.6958  51.8625  81.8583  56.4958  39.       7.4958
   7.7958   7.55    12.      93.5     21.      12.35    16.1     78.2667
  35.      55.9      7.8     30.      18.      26.2875  83.475   59.4
  77.9583  28.7125  56.9292   7.65    22.358

In [6]:
def get_features(data=data_train):
  Survived=data["Survived"].values
  Pclass=data["Pclass"].values
  Sex=data["Sex"].values
  SibSp=data["SibSp"].values
  Parch=data["Parch"].values
  Embarked=data["Embarked"].values
  Fare=data["Fare"].values
  return Survived, Pclass, Sex, SibSp, Parch, Embarked,Fare

Survived_train, Pclass_train, Sex_train, SibSp_train, Parch_train,Embarked_train,Fare_train=get_features()
Survived_test, Pclass_test, Sex_test, SibSp_test, Parch_test,Embarked_test,Fare_test=get_features(data=data_test)



#Muinomial

In [7]:
data.head()

,PassengerId,Survived,Pclass,Name,Sex,SibSp,Parch,Ticket,Fare,Embarked
0,1,0,3,"Braund, Mr. Owen Harris",male,1,0,A/5 21171,7.2500,S
1,2,1,1,"Cumings, Mrs. John Bradley (Florence Briggs Th...",female,1,0,PC 17599,71.2833,C
2,3,1,3,"Heikkinen, Miss. Laina",female,0,0,STON/O2. 3101282,7.9250,S
3,4,1,1,"Futrelle, Mrs. Jacques Heath (Lily May Peel)",female,1,0,113803,53.1000,S
4,5,0,3,"Allen, Mr. William Henry",male,0,0,373450,8.0500,S


P(Survived)*P(X|Survived)

P(notSurvived)*P(X|notSurvived)

In [8]:
#Xác xuất tiên nghiệm. 
P_Survived=0.5
P_notSurvived=0.5
#Sex
##Survived
P_SexisMale_Survived= np.sum((Sex_train[Survived_train==1]=="male"))/np.sum((Survived_train==1))
P_SexisFemale_Survived=np.sum((Sex_train[Survived_train==1]=="female"))/np.sum((Survived_train==1))
##Tính xác xuất hậu nghiệm trên tập (P(X|Survived) trên tập test)
P_Sex_Survived= P_SexisMale_Survived*(Sex_test=="male").astype(float) +P_SexisFemale_Survived*(Sex_test=="female").astype(float)

##notSurvivied
P_SexisMale_notSurvived= np.sum((Sex_train[Survived_train==0]=="male"))/np.sum((Survived_train==0))
P_SexisFemale_notSurvived=np.sum((Sex_train[Survived_train==0]=="female"))/np.sum((Survived_train==0))
##Tính xác xuất hậu nghiệm trên tập (P(X|notSurvived) trên tập test)
P_Sex_notSurvived= P_SexisMale_notSurvived*(Sex_test=="male").astype(float) +P_SexisFemale_notSurvived*(Sex_test=="female").astype(float)

In [10]:
#tích P(Survived|X)*P(Survived)
P_Survived_combine=P_Survived*P_Sex_Survived
#tích P(notSurvived|X)*P(notSurvived)
P_notSurvived_combine=P_notSurvived*P_Sex_notSurvived


In [13]:
np.mean((P_Survived_combine>P_notSurvived_combine).astype(float)==Survived_test)

0.797752808988764

In [ ]:
#Kết quả
np.mean((P_Survived_combine>P_notSurvived_combine).astype(float)==Survived_test)

In [14]:

## Dùng sklearn
from sklearn.naive_bayes import CategoricalNB
Sex_train1= (Sex_train.reshape(-1,1) == 'male').astype(float)
Sex_test1= (Sex_test.reshape(-1,1) == 'male').astype(float)
nb = CategoricalNB(alpha=1.0e-10).fit(Sex_train1, Survived_train)

nb.score(Sex_test1,Survived_test)

0.797752808988764

#Gaussian
$$p(x_i|c)=P(x_i|\mu_{ci},\sigma_{ci}^2)=\frac{1}{\sqrt{2\pi\sigma_{ci}^2}}exp(-\frac{(x_i-\mu_{ci})^2}{2\sigma_{ci}^2})$$

In [17]:


## tính tham số
Sibsp_mean_Survived = np.mean(SibSp_train[Survived_train==1])
Sibsp_std_Survived = np.std(SibSp_train[Survived_train==1])

Sibsp_mean_notSurvived = np.mean(SibSp_train[Survived_train==0])
Sibsp_std_notSurvived = np.std(SibSp_train[Survived_train==0])

def normal(x, mean, std):
    return 1/(np.sqrt(2*np.pi)*std)*np.exp(-np.square(x-mean)/(2*np.square(std)))

## P(X|Sur) và P(X|notSur)
P_SibSp_Survived = normal(SibSp_test, Sibsp_mean_Survived , Sibsp_std_Survived )
P_SibSp_notSurvived = normal(SibSp_test, Sibsp_mean_notSurvived, Sibsp_std_notSurvived)


P_Survived_combine=P_Survived*P_SibSp_Survived
P_notSurvived_combine= P_notSurvived*P_SibSp_notSurvived

print(np.mean((P_Survived_combine>P_notSurvived_combine).astype(int)==Survived_test))

## Dùng sklearn
from sklearn.naive_bayes import GaussianNB
nb = GaussianNB().fit(SibSp_train.reshape(-1,1), Survived_train)
print(np.mean(Survived_test == nb.predict(SibSp_test.reshape(-1,1))))


0.38764044943820225
0.3707865168539326


Bài tập
1. Dùng các features: Sex, Pclass, và Embarked để xây dựng thuật toán Naive Bayes Multinomial. (4đ)
2. Dùng các features: SibSp,	Parch, và Fare để xây dựng thuật toán Naive Bayes Gaussian (Khỏi normalize). (4đ)
3. Từ bộ dữ liệu liên minh hãy xây dựng thuật toán Naive Bayes bằng cách dùng tất cả features (mix cả Multinomial và Gaussian. Lưu ý: Khỏi chia tập train, test. Dự đoán thẳng trên tập train) (2đ)

In [15]:
#Chuẩn bị data cho câu 3
import pandas as pd
df = pd.read_csv("https://raw.githubusercontent.com/dinhvietcuong1996/Lab-MachineLearningCourse/master/Lab04/lienminh.csv")
df.head()

,killsDiff,minionsKilledDiff,wardPlacedDiff,firstBlood,heralds,dragons,teamWins
0,3,-2,13,blue,none,none,red
1,0,-66,0,red,red,red,red
2,-4,-17,0,red,none,blue,red
3,-1,-34,28,red,blue,none,red
4,0,-15,58,red,none,red,red


Về cơ bản, Liên Minh Huyền Thoại là một game giữa hai đội Blue và Red với mục tiêu là phá hủy nhà chính đối phương. Mỗi đội có 5 người chơi, mỗi người điều khiển 1 nhân vật. Trong trò chơi, để chiến thắng người chơi có thể:

Hạ gục đối phương: thể hiện bằng chỉ số hạ gục (Kills)

Hạ gục lính kiếm tiền mua trang bị (MinionsKilled)

Cắm mắt để có tầm nhìn, thêm thông tin về đối phương (WardsPlaced)

Ăn sứ giả lấy bùa và vật phẩm (Heralds)

Ăn bùa rồng (Dragons)

Mạng đầu tiên (firstBlood)

Đội thắng (teamWins)

In [16]:
#3
killsDiff = df['killsDiff'].values
minionsKilledDiff = df['minionsKilledDiff'].values
wardPlacedDiff = df['wardPlacedDiff'].values
firstBlood = df['firstBlood'].values
heralds = df['heralds'].values
dragons = df['dragons'].values

teamWins = df['teamWins'].values

P_blueWins=0.5
P_redWins=0.5